In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

C:\Users\Nishu\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
train = pd.read_csv('train.csv/train.csv')
test = pd.read_csv('test.csv/test.csv')

In [5]:
def prepare_data(df,n_cell_x,n_cell_y):
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
    eps = 0.00001
    
    ix = np.where(df.x.values < eps, 0, df.x.values - eps)
    iy = np.where(df.y.values < eps, 0, df.y.values - eps)
    
    pos_x = (ix / size_x).astype(int)
    pos_y = (iy / size_y).astype(int)
    
    df['grid_cell'] = pos_y * n_cell_x + pos_x
    
    ##This way we have created another column as grid_cell in our original train&test dataset
    initial_date = np.datetime64('2013-01-01T00:00',dtype = 'datetime64[m]')
    d_time = pd.DatetimeIndex(initial_date + np.timedelta64(int(m),'m') for m in df.time.values)
    
    ##Now extract all components of time from datetime index as save it as a new column in train & test data-set
    
    df['hour'] = d_time.hour
    df['minute'] = d_time.minute
    df['second'] = d_time.second
    df['year'] = d_time.year
    df['week'] = d_time.week
    df['weekofyear'] = d_time.weekofyear
    df['weekday'] = d_time.weekday
    
    ##Remove the time variable as feature generation from time is already done
    df.drop('time',axis=1,inplace=True)
    
    return(df) 

In [6]:
def prepare_grid(df_train,df_test,gid):
    df_train=df_train.loc[df_train.grid_cell == gid]
    df_test=df_test.loc[df_test.grid_cell == gid]
    row_ids = df_test.row_id.values
    le = LabelEncoder()
    y = le.fit_transform(df_train.place_id.values)
        
    clf = KNeighborsClassifier(n_neighbors=15,metric = 'manhattan', weights='distance')
        
    df_train.drop(['place_id','grid_cell'],axis=1,inplace=True)
    df_test.drop(['grid_cell'], axis=1,inplace=True)
        
    clf.fit(df_train,y)
        
    preds = clf.predict_proba(df_test)
        
    labels = le.inverse_transform(np.argsort(preds,axis=1)[:,::-1][:,:3])
        
    return(labels,row_ids)        

In [7]:
def process_grids(train,test):
    n_cell_x = 20
    n_cell_y = 40
    
    train=prepare_data(train,n_cell_x,n_cell_y)
    test=prepare_data(test,n_cell_x,n_cell_y)
    
    prediction = np.zeros((test.shape[0],3),dtype=int)
    
    labels,row_ids = prepare_grid(train,test,100)
    prediction[row_ids] = labels
    
    temp = pd.DataFrame(prediction,dtype=str, columns=['id0', 'id1','id2'])
    result = temp.id0.str.cat([temp.id1,temp.id2],sep = " ")
    
    result.name = 'place_id'
    
    result.to_csv('submit_knn_grid01.csv', index_label= 'row_id', index = True, header = True)  

In [8]:
process_grids(train,test)

C:\Users\Nishu\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Nishu\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# In-efficient way of computing the predictions
c=0
for gid in train.grid_cell.values:
    df_train = train.loc[train.grid_cell == gid,:]
    df_test  = test.loc[test.grid_cell == gid,:]
    le = LabelEncoder()
    list[c] = pd.DataFrame(columns=['row_id','place_id'])
    list[c]['row_id'] = df_test.row_id.values
    y = le.fit_transform(df_train.place_id.values)
    df_train.drop(['place_id','grid_cell'],axis=1,inplace=True)
    df_test.drop('grid_cell',axis=1,inplace=True)

    clf = KNeighborsClassifier(n_neighbors=10,weights='distance',metric = 'euclidean')

    clf.fit(df_train,y)
    preds = clf.predict_proba(df_test)
    d = 0
    for m in preds:
        a = np.argsort(m)[::-1][0:3]
        
        tgt = le.inverse_transform(y)
        
        tgt_place_id = np.take(tgt,[a[0],a[1],a[2]])
        
        list[c].loc[d,'place_id'] = str(tgt_place_id[0]) + " " + str(tgt_place_id[1]) + " " + str(tgt_place_id[2])
        d  = d+1
        c  = c+1